In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import xlrd
import lxml
from time import sleep
from xlsxwriter import Workbook
import requests


#--------------------------Search-------------------------
def search(keyword='Resturants',where='Chicago'):
    driver=webdriver.Chrome('chromedriver')
    url='https://www.yelp.com'
    driver.get(url)
    sleep(1)
    driver.find_element_by_id('find_desc').send_keys(keyword)
    btn=driver.find_element_by_id('dropperText_Mast')
    btn.clear()
    btn.send_keys(where)
    btn.submit()
    url=driver.current_url
    driver.close()
    return url

#------------------------------filtered links--------------------------------



def get_filtered(url):
    driver=webdriver.Chrome('chromedriver')
    c=0
    flag=True
    filterd=[]
    count=0
    end=5
    condition=True
    while flag:
        driver.get(url)
        btn=driver.find_element_by_xpath('//span[@class="lemon--span__373c0__3997G filter__373c0__wBU2H"]')
        btn.click()
        sleep(2)
        btn=driver.find_element_by_xpath('//button[@class="link__373c0__29O4k"]//p[@class="lemon--p__373c0__3Qnnj text__373c0__2pB8f text-color--inherit__373c0__w_15m text-align--left__373c0__2pnx_ text-size--small__373c0__3SGMi"]')
        btn.click()
        sleep(1)
        div=driver.find_element_by_xpath('//div[@class="lemon--div__373c0__1mboc u-padding-t4 u-padding-r6 u-padding-b6 u-padding-l6 border-color--default__373c0__2oFDT"]')
        inputslis=div.find_elements_by_tag_name('input')
        search=driver.find_element_by_xpath('//button[@class="button__373c0__3yl_n small__373c0__1sSnK primary__373c0__oJujF"]')
        length=len(inputslis)
        if condition:
            print('length',length)
            condition=False

                        #x  x+5
        for i in range(count,end):

            try:
                if count==length:
                    break
                inputslis[count].click()
                print('count=',count)
                count+=1
                c+=1

            except:
                print(" Error")
                flag=False
                print('c=',c)

        end+=5

        search.click()
        sleep(5)
        filterd.append(driver.current_url)
        if count==length:
            break
    driver.close()



    return list(set(filterd))



#----------------------------------get listing links----------------------

class Source:
    def __init__(self,source):
        self.source=source
        self.final=[]
        self.get_links()
    def no_of_results(self,link):
        res0=requests.get(link)
        soup0=BeautifulSoup(res0.content,'lxml')
        soup0.find
        result0=soup0.find('h1',class_="lemon--h1__373c0__2ZHSL heading--h2__373c0__1TQtb alternate__373c0__1uacp")
        result0=result0.next_sibling.text
        string=result0
        f_start=string.find('f')
        nor=int(string[f_start+2:])
        _=string.find('-')
        per_p=int(string[_+1:f_start-1])
        return [nor,per_p]
    def get_links(self):
        for inde,link in enumerate(self.source):
            print(('Source\t'+str(inde)).center(100,'-'))
            datip=self.no_of_results(link)
            nor=datip[0]
            per_p=datip[1]
            print('No. of results:',nor)
            print('Required Itterations:',int(nor/per_p))
            c=0
            countt=0
            while c<=nor:
                link=self.source[inde]
                print('----------------------------itteration:',int(c/per_p))
                print('c  =',c)
                if c==0:
                    link=link
                else:
                    link=link+"&start="+str(c)

                response=requests.get(link)
                soup=BeautifulSoup(response.content,'lxml')
                try:
                    resturants=soup.find_all('div',class_="lemon--div__373c0__1mboc u-padding-t3 u-padding-b3 border--top__373c0__19Owr border-color--default__373c0__2oFDT")
                    print('B_res:',len(resturants))
                except:
                    print('Failed to find resturants')
                    resturants=[]
                try:
                    first=list(soup.find_all('div',class_="lemon--div__373c0__1mboc u-padding-t3 u-padding-b3 border-color--default__373c0__2oFDT")[-1])
                except:
                    print('Failed to find first')
                    first=[]
                resturants=first+resturants
                countt+=len(resturants)
                print('Got:',len(resturants))
                print('Total:',countt)
                gog=['https://www.yelp.com'+div.find('a')['href'] for div in resturants]
                self.final+=gog
                c+=per_p
#------------------------------------------------------------------------------------------------
what=input("Enter search keywords:")
where=input('Enter Location:')
url=search(what,where)
source=get_filtered(url)
get=Source(source)
k=get.final
uniq=set(k)


#----------------------------------------------------------------------------------------------

workbookin=Workbook('stored_data1.xlsx')
worksheetin=workbookin.add_worksheet()

rowno=0
links=list(uniq)
driver=webdriver.Chrome('chromedriver')
for index,link in enumerate(links):
    print('processing link:',index+1)
    driver.get(link)
    sleep(2)
    try:
        btn=driver.find_element_by_partial_link_text('More Attributes')
        btn.click()
    except:
        print('No "More Attributes" Button found')
        pass
    soup=BeautifulSoup(driver.page_source,'lxml')
    try:
        print('getting name')
        name=soup.find('h1').text
    except:
        print('name not found')
        name=None
    try:
        print('getting category')
        maybe=soup.find('span',class_="lemon--span__373c0__3997G display--inline__373c0__1DbOG u-space-r1 border-color--default__373c0__2oFDT").text
        if maybe.find('$') != -1:
            span=soup.find('span',class_="lemon--span__373c0__3997G display--inline__373c0__1DbOG u-space-r1 border-color--default__373c0__2oFDT").next_sibling
            span=span.find_all('span')
            category=''.join([span[i].text for i in range(len(span))])
        else:
            category=maybe
    except:
        print('category not found')
        category=None
    try:
        print('getting adress')
        spans=soup.find('address',class_="lemon--address__373c0__2sPac").find_all('span')
        adress=''
        for span in spans:
            adress+=span.text
    except:
        print('adress not found')
        adress=None
    try:
        print('getting telephone number')
        span=soup.find_all('p',class_="lemon--p__373c0__3Qnnj text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_")
        tel=span[-2].text
        tele=(tel.replace('(','').replace(')',''))
        int(tele.replace(' ','').replace('-',''))
    
    except:
        print('telephone number not found')
        tel=None

    try:
        print('getting attributes')
        attribute=[]
        for i in range(len(section)):
            try:
                attribute.append(section[i].find('div',class_="lemon--div__373c0__1mboc u-space-b2 arrange__373c0__UHqhV vertical-align-middle__373c0__2TQsQ border-color--default__373c0__2oFDT").next_sibling.find('div',class_="lemon--div__373c0__1mboc border-color--default__373c0__2oFDT").find('div',class_="lemon--div__373c0__1mboc u-space-b3 border-color--default__373c0__2oFDT").find('div',class_="lemon--div__373c0__1mboc arrange__373c0__UHqhV gutter-12__373c0__3kguh layout-wrap__373c0__34d4b layout-2-units__373c0__3CiAk border-color--default__373c0__2oFDT"))
            except:
                pass
        attribute=attribute[0]
        attribute=attribute.find_all('div',class_="lemon--div__373c0__1mboc arrange-unit__373c0__1piwO border-color--default__373c0__2oFDT")
        attributes=[attribute[i].div for i in range(len(attribute)) if attribute[i].div is not None ]
        attr_dict=[]
        for i in range(len(attributes)):
            div=attributes[i].div.next_sibling
            attr_dict.append({div.span.text:div.span.next_sibling.text[1:]})
        
    except:
        print('attributes not found')
        attr_dict=None
    try:
        print('getting timings')
        section=soup.find_all('section',class_="lemon--section__373c0__fNwDM u-space-t4 u-padding-t4 border--top__373c0__19Owr border-color--default__373c0__2oFDT")
        for i in section:
            try:
                sect=i.div.next_sibling.div.next_sibling.find('div',class_="lemon--div__373c0__1mboc border-color--default__373c0__2oFDT").find('div',class_="lemon--div__373c0__1mboc border-color--default__373c0__2oFDT")
            except:
                pass
        tr=sect.table.tbody.find_all('tr')
        time_dict=[]
        for i in range(len(tr)):
            time_dict.append({tr[i].th.p.text:tr[i].td.ul.li.p.text})
        try:
            if list(attr_dict[0].keys())[0]=='Health Score':
                score=list(attr_dict[0].values())[0]
                attr_dict.pop(0)
            else:
                score=None

        except:
            score=None

    except:
        print('timings not found')
        time_dict=None
    try:
        print('getting claimed?')
        claimed=soup.find('h1').next_sibling.span.text
    except:
        print('claimed? not found')
        claimed=None
    try:
        print('getting reviews')
        reviews=soup.find('h1').parent.next_sibling.div.next_sibling.p.text
    except:
        print('reviews not found')
        reviews= None
    try:
        print('getting website address')
        span=soup.find_all('p',class_="lemon--p__373c0__3Qnnj text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_")
        a=span[-2].parent.parent.parent.previous_sibling.div.div.next_sibling.a.text
        site='https://www.'+a
    except:
        print('website address not found')
        site=None
    try:
        print('getting ratings')
        rating=soup.find('h1').parent.next_sibling.div.span.div['aria-label']
        end=rating.find('r')+1
        rating=rating[:end]
    except:
        print('ratings not found')
        rating=None
    for coulmn,item in enumerate([name,category,adress,tel,score,str(attr_dict),str(time_dict),rating,claimed,reviews,site,link]):
        worksheetin.write(rowno,coulmn,item)
    
    
        
        
        
        
    rowno+=1 
    #sleep(8)
    
print('closing.....')
driver.close()
workbookin.close()
